In [11]:
print('Started...')
import glob
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import png
import time
import numpy as np
import os
from fnmatch import fnmatch


# Gobals
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/malignant/1.2.840.113681.2230565232.95*'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/normal/1.2.840.113681.2230565232.95*'
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/15'
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/2D/6mm/'

# File walk method

fMal = []
for path, subdirs, files in os.walk(DICOM_FILES):
    for name in files:
        if fnmatch(name, '*.dcm'):
            fMal.append(os.path.join(path, name))
            
print(len(fMal), ' dicom images found')


# Load dicom files into np array
malignant = np.array([])
dicomImg = {'malignant':np.array([]), 'normal':np.array([])}
count = 0
print(len(fMal), ' Malignant found')
print('Loading images...')
for f in fMal:
    dicomImg['malignant'] = np.append(dicomImg['malignant'], pydicom.dcmread(f))
    count += 1
    print(count, '/', len(fMal))


# __________________________________________
print(dicomImg['malignant'].shape)
# print(dicomImg['malignant'][0].shape)
print(dicomImg['malignant'][0])
#print(dicomImg['malignant'])
print('name:\n', dicomImg['malignant'][0].PresentationIntentType)

forProcessingCount = 0
for _ in dicomImg['malignant']:
    tmp = _.PresentationIntentType
    print(tmp)
    if tmp == 'FOR PROCESSING':
        forProcessingCount += 1
print('\nTotal DICOM: ', len(dicomImg['malignant']))
print('Total FOR PROCESSING: ', forProcessingCount)

    









   
# Import xls file
xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch_50_IMAGE.xls')
sheet = xls.parse(1)

# Create a dict where the key is the image name
# Each key has the image, and coords
img = {}
for i in range(len(fMal)):
    key = os.path.basename(fMal[i])[:-4]
    # Find row in the xml file that holds the img info
    try:
        indx = [_==key for _ in sheet['ReferencedSOPInstanceUID']].index(True) # ImageSOPIUID, ReferencedSOPInstanceUID
        img.update({key:{}})
        img[key].update({'img': dicomImg['malignant'][i].pixel_array})
        img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
        img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})
    except ValueError:
        print('MY_ERROR: key not found:\n', key)
        
print(len(img), 'DICOM images extracted')
# print(img)
# Crop the images to given ROI
toDelete = [] # Keep track of error causing keys and delete after loop
for key in img:
    try:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        x = [int(x[0]), int(x[1])]
        y = [int(y[0]), int(y[1])]
        #x is width, y is height
        #in numpy array, y,x
        img[key].update({'cropROI':tmp[y[0]:y[1], x[0]:x[1]]}) 
    except:
        print('ROI extraction failed...Removing key\nkey:  ',key, '\nx  :', x,'\ny:  ',y)
        toDelete.append(key)
    
#img_calc = img['1.2.840.113681.2230565232.954.3504500766.32'] 
for _ in toDelete:
    img.pop(_)


if 1==2: #Write images to disk with markers and basic crop
    for key in img:
        plt.figure(figsize=(20,20))
        marker = [(img[key]['x'][1] + img[key]['x'][0])/2, (img[key]['y'][1] + img[key]['y'][0])/2 ]   
        plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
        plt.plot(marker[0], marker[1], marker='x', color=[1,0,1], markersize=30)
        plt.savefig('./marked/normal/' + key +'_full.png')
        #plt.show()
        plt.close()

        plt.figure(figsize=(20,20))
        plt.imshow(img[key]['cropROI'], cmap='gray') 
        plt.savefig('./marked/normal/' + key +'_crop.png')   
        #plt.show()
        plt.close()

# Crop the images so that the ROI is centred but all crops are the same size
if 1 == 1:
    cS = 256
    for key in img:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        c = [round((x[0]+x[1])/2), round((y[0]+y[1])/2)]
        # Pad images before cropping (wrap around)
        pad = 1000
        tmp = np.pad(tmp, pad, mode='wrap')
        img[key].update({'crop': tmp[int(c[1]-cS/2+pad):int(c[1]+cS/2+pad), int(c[0]-cS/2+pad):int(c[0]+cS/2+pad)]})
        # Reshape from (256, 256) to (256, 256, 1)
        img[key]['crop'] = np.reshape(img[key]['crop'],(img[key]['crop'].shape[0], img[key]['crop'].shape[1], 1))
    
# Find bit depth
if 1 == 2:
    maxmax = 0
    for key in img:
        tmp = img[key]['img']
        print(np.amax(tmp))
        if np.amax(tmp) > maxmax:
            maxmax = np.amax(tmp)
    print('The largest value is: ', maxmax)
    
# Find average ROI size
if 1 == 2:
    totalX = 0
    totalY = 0
    for key in img:
        x = img[key]['x']
        y = img[key]['y']
        totalX += x[1] - x[0]
        totalY += y[1] - y[0]
    print('Average ROI width: ', totalX/len(img), '\nAverage ROI length: ', totalY/len(img))
    
# View crops / save to disk
if 1 == 2:
    count = 0
    for key in img:
        if count == 0:
            print(img[key]['crop'])

        count+=1
        f = open('./crop256/normal/' + key + '.png', 'wb')
        w = png.Writer(width = cS, height = cS, bitdepth=16, greyscale=True)
        w.write(f, img[key]['crop'])
        f.close()
        print(count, '/', len(img))
        if count == -1:
            break




print('Done')

Started...
2396  dicom images found
2396  Malignant found
Loading images...
1 / 2396
2 / 2396
3 / 2396
4 / 2396
5 / 2396
6 / 2396
7 / 2396
8 / 2396
9 / 2396
10 / 2396
11 / 2396
12 / 2396
13 / 2396
14 / 2396
15 / 2396
16 / 2396
17 / 2396
18 / 2396
19 / 2396
20 / 2396
21 / 2396
22 / 2396
23 / 2396
24 / 2396
25 / 2396
26 / 2396
27 / 2396
28 / 2396
29 / 2396
30 / 2396
31 / 2396
32 / 2396
33 / 2396
34 / 2396
35 / 2396
36 / 2396
37 / 2396
38 / 2396
39 / 2396
40 / 2396
41 / 2396
42 / 2396
43 / 2396
44 / 2396
45 / 2396
46 / 2396
47 / 2396
48 / 2396
49 / 2396
50 / 2396
51 / 2396
52 / 2396
53 / 2396
54 / 2396
55 / 2396
56 / 2396
57 / 2396
58 / 2396
59 / 2396
60 / 2396
61 / 2396
62 / 2396
63 / 2396
64 / 2396
65 / 2396
66 / 2396
67 / 2396
68 / 2396
69 / 2396
70 / 2396
71 / 2396
72 / 2396
73 / 2396
74 / 2396
75 / 2396
76 / 2396
77 / 2396
78 / 2396
79 / 2396
80 / 2396
81 / 2396
82 / 2396
83 / 2396
84 / 2396
85 / 2396
86 / 2396
87 / 2396
88 / 2396
89 / 2396
90 / 2396
91 / 2396
92 / 2396
93 / 2396
94 

751 / 2396
752 / 2396
753 / 2396
754 / 2396
755 / 2396
756 / 2396
757 / 2396
758 / 2396
759 / 2396
760 / 2396
761 / 2396
762 / 2396
763 / 2396
764 / 2396
765 / 2396
766 / 2396
767 / 2396
768 / 2396
769 / 2396
770 / 2396
771 / 2396
772 / 2396
773 / 2396
774 / 2396
775 / 2396
776 / 2396
777 / 2396
778 / 2396
779 / 2396
780 / 2396
781 / 2396
782 / 2396
783 / 2396
784 / 2396
785 / 2396
786 / 2396
787 / 2396
788 / 2396
789 / 2396
790 / 2396
791 / 2396
792 / 2396
793 / 2396
794 / 2396
795 / 2396
796 / 2396
797 / 2396
798 / 2396
799 / 2396
800 / 2396
801 / 2396
802 / 2396
803 / 2396
804 / 2396
805 / 2396
806 / 2396
807 / 2396
808 / 2396
809 / 2396
810 / 2396
811 / 2396
812 / 2396
813 / 2396
814 / 2396
815 / 2396
816 / 2396
817 / 2396
818 / 2396
819 / 2396
820 / 2396
821 / 2396
822 / 2396
823 / 2396
824 / 2396
825 / 2396
826 / 2396
827 / 2396
828 / 2396
829 / 2396
830 / 2396
831 / 2396
832 / 2396
833 / 2396
834 / 2396
835 / 2396
836 / 2396
837 / 2396
838 / 2396
839 / 2396
840 / 2396
841 / 2396

1459 / 2396
1460 / 2396
1461 / 2396
1462 / 2396
1463 / 2396
1464 / 2396
1465 / 2396
1466 / 2396
1467 / 2396
1468 / 2396
1469 / 2396
1470 / 2396
1471 / 2396
1472 / 2396
1473 / 2396
1474 / 2396
1475 / 2396
1476 / 2396
1477 / 2396
1478 / 2396
1479 / 2396
1480 / 2396
1481 / 2396
1482 / 2396
1483 / 2396
1484 / 2396
1485 / 2396
1486 / 2396
1487 / 2396
1488 / 2396
1489 / 2396
1490 / 2396
1491 / 2396
1492 / 2396
1493 / 2396
1494 / 2396
1495 / 2396
1496 / 2396
1497 / 2396
1498 / 2396
1499 / 2396
1500 / 2396
1501 / 2396
1502 / 2396
1503 / 2396
1504 / 2396
1505 / 2396
1506 / 2396
1507 / 2396
1508 / 2396
1509 / 2396
1510 / 2396
1511 / 2396
1512 / 2396
1513 / 2396
1514 / 2396
1515 / 2396
1516 / 2396
1517 / 2396
1518 / 2396
1519 / 2396
1520 / 2396
1521 / 2396
1522 / 2396
1523 / 2396
1524 / 2396
1525 / 2396
1526 / 2396
1527 / 2396
1528 / 2396
1529 / 2396
1530 / 2396
1531 / 2396
1532 / 2396
1533 / 2396
1534 / 2396
1535 / 2396
1536 / 2396
1537 / 2396
1538 / 2396
1539 / 2396
1540 / 2396
1541 / 2396
1542

2145 / 2396
2146 / 2396
2147 / 2396
2148 / 2396
2149 / 2396
2150 / 2396
2151 / 2396
2152 / 2396
2153 / 2396
2154 / 2396
2155 / 2396
2156 / 2396
2157 / 2396
2158 / 2396
2159 / 2396
2160 / 2396
2161 / 2396
2162 / 2396
2163 / 2396
2164 / 2396
2165 / 2396
2166 / 2396
2167 / 2396
2168 / 2396
2169 / 2396
2170 / 2396
2171 / 2396
2172 / 2396
2173 / 2396
2174 / 2396
2175 / 2396
2176 / 2396
2177 / 2396
2178 / 2396
2179 / 2396
2180 / 2396
2181 / 2396
2182 / 2396
2183 / 2396
2184 / 2396
2185 / 2396
2186 / 2396
2187 / 2396
2188 / 2396
2189 / 2396
2190 / 2396
2191 / 2396
2192 / 2396
2193 / 2396
2194 / 2396
2195 / 2396
2196 / 2396
2197 / 2396
2198 / 2396
2199 / 2396
2200 / 2396
2201 / 2396
2202 / 2396
2203 / 2396
2204 / 2396
2205 / 2396
2206 / 2396
2207 / 2396
2208 / 2396
2209 / 2396
2210 / 2396
2211 / 2396
2212 / 2396
2213 / 2396
2214 / 2396
2215 / 2396
2216 / 2396
2217 / 2396
2218 / 2396
2219 / 2396
2220 / 2396
2221 / 2396
2222 / 2396
2223 / 2396
2224 / 2396
2225 / 2396
2226 / 2396
2227 / 2396
2228

FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PROCESSING
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PRESENTATION
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PROCESSING
FOR PROCESSING
FOR PROCESSING
FOR PRESENTATION
FOR PRESENTATION
FOR PRESENTATION
FOR PROCESSING
FOR PROCESSING
FOR PRESEN

...

MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3010.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3019.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3015.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3022.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3011.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3014.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3018.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3023.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3221.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3226.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.3233.0

 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4647.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4660.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4648.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4656.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4651.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.2581.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.2577.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.2579.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.2575.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.5060.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.5067.0
MY_ERROR: key not found:

 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.5275.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.5269.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.5287.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4987.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4980.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4979.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4975.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4984.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4983.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4988.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.4976.0
MY_ERROR: key not found:

 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.721.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.746.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.726.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.706.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.670.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.677.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.669.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.665.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.666.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.673.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.678.0
MY_ERROR: key not found:

 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.5759.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.1474672.1549.1517598161068.5752.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.647.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.655.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.652.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.659.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.662.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.649.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.645.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.651.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.14746728.1549.1517598161068.641.0
MY_ERROR: key not found: